In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)
    
import utils
from api.gui.utils import tables
from api.gui.payoff import Form
from api.data.processing import get_current_data

In [2]:

data = get_current_data()

In [3]:
g = data.groupby(['capital_stack', 'maturity_bucket', 'custom_rating', 'currency'])

In [4]:
for index, group in g:
    if len(group) > 5:
        break

In [5]:
ref_fields = [
    'ticker',
    'name',
    'isin',
    'currency',
    'bics_industry',
    'workout_date',
    'calc_maturity',
    'maturity_bucket',
    'custom_rating',
    'amount_outstanding',
    'issue_date',       
]

In [6]:
mkt_fields = [
    'z_spread',
    'z_spread_change_12m',
    'z_spread_change_ytd',
    'z_spread_change_6m',
    'z_spread_change_3m',
    'z_spread_change_1m',
    'z_spread_change_1w'
]

In [7]:
group[
    [
        *ref_fields,
        *mkt_fields,
    ]
].dropna(subset=mkt_fields)

ticker               name          isin currency  \
security                                                          
AX329820 Corp   ACAFP   ACAFP 6 7/8 PERP  US225313AL91      USD   
ZR276827 Corp   SWEDA   SWEDA 5 5/8 PERP  XS2046625765      USD   
EK478624 Corp    HSBC    HSBC 6 3/8 PERP  US404280AS86      USD   
AO937605 Corp  BAERVX  BAERVX 4 3/4 PERP  XS1679216801      USD   

                    bics_industry workout_date calc_maturity maturity_bucket  \
security                                                                       
AX329820 Corp   Diversified Banks   2024-09-23    2171-03-23             2yr   
ZR276827 Corp               Banks   2024-09-17    2171-03-17             2yr   
EK478624 Corp   Diversified Banks   2024-09-17    2171-03-17             2yr   
AO937605 Corp  Financial Services   2024-09-12    2171-03-12             2yr   

              custom_rating  amount_outstanding issue_date    z_spread  \
security                                                                 
AX329820 Corp          BBB-        1.250000e+09 2019-02-27  475.866471   
ZR276827 Corp          BBB-        5.000000e+08 2019-08-29  413.998061   
EK478624 Corp          BBB-        2.250000e+09 2014-09-17  475.357290   
AO937605 Corp          BBB-        3.000000e+08 2017-09-12  514.589085   

               z_spread_change_12m  z_spread_change_ytd  z_spread_change_6m  \
security                                                                      
AX329820 Corp           239.649909           226.014044          209.389744   
ZR276827 Corp           182.468781           206.849381          195.438681   
EK478624 Corp           233.457584           203.132884          194.417684   
AO937605 Corp           241.222278           199.912478          174.502178   

               z_spread_change_3m  z_spread_change_1m  z_spread_change_1w  
security                                                                   
AX329820 Corp          145.545044           40.629344           41.633744  
ZR276827 Corp           68.364881            2.320081           38.666881  
EK478624 Corp          107.336384           25.117484           33.357584  
AO937605 Corp         -101.518322           73.770878           40.408078

In [ ]:
group.iloc[:, :20]

ticker               name          isin currency country  \
security                                                                  
AX727792 Corp     BNP     BNP 6 5/8 PERP  US05565AHN63      USD      FR   
AX329820 Corp   ACAFP   ACAFP 6 7/8 PERP  US225313AL91      USD      FR   
ZR276827 Corp   SWEDA   SWEDA 5 5/8 PERP  XS2046625765      USD      SE   
EK478624 Corp    HSBC    HSBC 6 3/8 PERP  US404280AS86      USD      GB   
EK027149 Corp   ACAFP   ACAFP 7 7/8 PERP  US225313AD75      USD      FR   
AO937605 Corp  BAERVX  BAERVX 4 3/4 PERP  XS1679216801      USD      CH   

              bics_sector       bics_industry  amount_outstanding issue_date  \
security                                                                       
AX727792 Corp  Financials   Diversified Banks        1.500000e+09 2019-03-25   
AX329820 Corp  Financials   Diversified Banks        1.250000e+09 2019-02-27   
ZR276827 Corp  Financials               Banks        5.000000e+08 2019-08-29   
EK478624 Corp  Financials   Diversified Banks        2.250000e+09 2014-09-17   
EK027149 Corp  Financials   Diversified Banks        1.750000e+09 2014-01-23   
AO937605 Corp  Financials  Financial Services        3.000000e+08 2017-09-12   

              maturity calc_maturity workout_date maturity_type  coupon  \
security                                                                  
AX727792 Corp      NaT    2170-09-25   2024-03-25     PERP/CALL   6.625   
AX329820 Corp      NaT    2171-03-23   2024-09-23     PERP/CALL   6.875   
ZR276827 Corp      NaT    2171-03-17   2024-09-17     PERP/CALL   5.625   
EK478624 Corp      NaT    2171-03-17   2024-09-17     PERP/CALL   6.375   
EK027149 Corp      NaT    2171-03-23   2024-01-23     PERP/CALL   7.875   
AO937605 Corp      NaT    2171-03-12   2024-09-12     PERP/CALL   4.750   

              coupon_type rating_sp rating_moody rating_fitch rating_bbg  \
security                                                                   
AX727792 Corp    VARIABLE      BBB-          Ba1          BBB       BBB-   
AX329820 Corp    VARIABLE      BBB-         Baa3          BBB       BBB-   
ZR276827 Corp    VARIABLE      BBB-          Ba1          BBB       BBB-   
EK478624 Corp    VARIABLE       NaN         Baa3          BBB       BBB-   
EK027149 Corp    VARIABLE      BBB-          NaN          BBB       BBB-   
AO937605 Corp    VARIABLE       NaN         Baa3          NaN        NaN   

              is_fixed  
security                
AX727792 Corp        N  
AX329820 Corp        N  
ZR276827 Corp        N  
EK478624 Corp        N  
EK027149 Corp        N  
AO937605 Corp        N

In [9]:
data[['rating_sp', 'rating_moody']][:60]

rating_sp rating_moody
security                            
ZR767554 Corp       NaN          Ba3
BS986526 Corp      BBB-           A3
BT457336 Corp       NaN          NaN
LW742768 Corp       BB+          NaN
BS378767 Corp       BBB         Baa2
AZ151993 Corp       BB+         Baa2
BR551854 Corp       BB+         Baa3
AM923387 Corp      BBB-         Baa2
EK795322 Corp      BBB+         Baa1
AQ180386 Corp         B          Ba3
EJ688366 Corp       BB+          Ba1
BV716218 Corp       NaN          Aa2
EJ810150 Corp       BB+          Ba1
EK480568 Corp       BB+          Ba1
LW042943 Corp       BB+         Baa2
AN433208 Corp       BB+         Baa2
BT566261 Corp       NaN          Ba3
AP138030 Corp       NaN          Ba3
BP548648 Corp         B          NaN
ZO971715 Corp      BBB+         Baa2
BK226167 Corp        B+          Ba1
BN390689 Corp       NaN          NaN
BR337425 Corp       NaN          Ba1
AX644490 Corp       NaN          Ba3
AU788856 Corp       BB-         Baa3
ZO703315 Corp       BBB         Baa2
AS097893 Corp       BB-          NaN
BS355458 Corp        B+          NaN
ZQ326977 Corp       NaN          Ba1
AX414531 Corp       BB+          Ba1
AS201488 Corp       NaN          Ba1
AN079036 Corp       BB-          Ba1
AX520038 Corp      BBB-          Ba1
BJ498818 Corp         B          Ba1
EK135837 Corp       BB-         Baa3
BK790507 Corp        B+          Ba2
AW940417 Corp        BB         Baa3
ZS152988 Corp       NaN           B2
BT306419 Corp      BBB-          NaN
ZQ464759 Corp      BBB-         Baa2
BN423073 Corp        BB          NaN
JK891237 Corp        BB          Ba3
BS476990 Corp       NaN          NaN
EJ861753 Corp       BBB         Baa2
ZP935314 Corp       CCC           B3
AZ321278 Corp      BBB+         Baa2
BG039176 Corp       CCC         Caa1
BJ595099 Corp       BBB         Baa2
ZP539697 Corp        B+          NaN
AM377864 Corp       BBB         Baa2
BM834500 Corp      BBB+         Baa2
AX751539 Corp       BBB          NaN
LW023774 Corp       BB+          Ba1
EK354842 Corp       BB+         Baa2
ZO616782 Corp        BB         Baa2
AZ566172 Corp       NaN          Ba1
EJ415238 Corp       BB+         Baa3
BR794437 Corp         B          NaN
BJ419101 Corp       BB+          Ba1
AQ219162 Corp       BB+          Ba1